# Stages

1.   Load data
2.   Check data
3.   Implement the model
4.   Evaluate the results

In [ ]:
import pandas as pd

In [ ]:
sheet_id = "1r6sqFGnPqPYsXL0BYVH60DAsXfHuT2w0"
sheet_name = "Sheet1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
data_main = pd.read_csv(url)
data_main.head()

,Unnamed: 0,Headline,Startup,Start_id,End_id,my_index,ne,org_by_index,test,spacy_anao
0,0,accedo raises us$17 million in funding,accedo,1,6,"[(0, 6)]",ORG,accedo,0,"('""accedo raises us$17 million in funding""', {..."
1,1,uniti scores £1 million funding target through...,uniti,1,5,"[(0, 5)]",ORG,uniti,0,"('""uniti scores £1 million funding target thro..."
2,2,"cannabis inhaler producer, syqe medical, raise...",syqe medical,28,12,"[(27, 39)]",ORG,syqe medical,0,"('""cannabis inhaler producer, syqe medical, ra..."
3,3,"alphonse's talents raises € 600,000",alphonse's talents,1,18,"[(0, 18)]",ORG,alphonse's talents,0,"('""alphonse\'s talents raises € 600,000 ""', {'..."
4,4,libon raises €1.8 million,libon,1,5,"[(0, 5)]",ORG,libon,0,"('""libon raises €1.8 million""', {'entities': [..."


# Function used for preparing the data

In [ ]:
def create_bert_data():
  bert_data = pd.DataFrame()
  tok = []
  words = []
  sentence_num = []
  for ind in data.index:
    for word in data['Headline'].iloc[ind].split():
      startup_name = data['Startup'].iloc[ind].split()
      if len(startup_name) > 1:
        if word in startup_name:
            if word == startup_name[0] :
              tok.append("B-ORG")
            elif word in startup_name:
              tok.append("I-ORG")
        else:
          tok.append("O")   
      elif word == startup_name[0]:
          tok.append("B-ORG")
      else:
          tok.append("O")
      words.append(word)
      sentence_num.append(ind)
  bert_data['words'] = words
  bert_data['sentence_id'] = sentence_num
  bert_data['labels'] = tok
  return bert_data

In [ ]:
sheet_id = "1PEqM-4ORnUFgKct-6nS_Hq1Osoau1kIxTtL6Q0-zM_8"
sheet_name = "bert-data"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
data = pd.read_csv(url)
data.head()

,words,sentence_id,labels
0,accedo,0,B-ORG
1,raises,0,O
2,us$17,0,O
3,million,0,O
4,in,0,O


In [ ]:
train_data = data[data.sentence_id < 1677]
validation = data[(data.sentence_id > 1676) & (data.sentence_id < 2237) ]
test_data = data[data.sentence_id <= 2237]

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 25.5MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 1.8MB 25.5MB/s 
     |████████████████████████████████| 3.3MB 46.5MB/s 
     |████████████████████████████████| 1.2MB 41.6MB/s 
     |████████████████████████████████| 7.8MB 47.8MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 245kB 55.5MB/s 
     |████████████████████████████████| 2.5MB 49.8MB/s 
     |████████████████████████████████| 133kB 55.6MB/s 
     |████████████████████████████████| 174kB 52.2MB/s 
     |████████████████████████████████| 133kB 53.2MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 112kB 50.0MB/s 
     |████████████████████████████████| 4.2MB 26.8MB/s 
     |████████████████████████████████| 245kB 56.0MB/s 
     |████████████████████████████████| 122kB 51.9MB

In [ ]:
import logging
from simpletransformers.ner import NERModel, NERArgs


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


# Configure the model
model_args = NERArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True

model = NERModel(
    "distilbert", "distilbert-base-uncased", args=model_args
)

# Train the model
model.train_model(train_data, eval_data=validation)

# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_data)

# Make predictions with the model
predictions, raw_outputs = model.predict([sample_text])

INFO:filelock:Lock 140629776727952 acquired on /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock


Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

INFO:filelock:Lock 140629776727952 released on /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock
INFO:filelock:Lock 140629778129680 acquired on /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

INFO:filelock:Lock 140629778129680 released on /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model)

...

INFO:simpletransformers.ner.ner_model: Training of distilbert model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/280 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.04777889508487923, 'precision': 0.9273897058823529, 'recall': 0.9299539170506912, 'f1_score': 0.9286700414173954}


NameError: ignored

In [ ]:
predictions, raw_outputs = model.predict([data_main.Headline[2789]])

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions

[[{'a': 'B-ORG'},
  {'cloud': 'I-ORG'},
  {'guru': 'I-ORG'},
  {'raised': 'O'},
  {'a': 'O'},
  {'$257': 'O'},
  {'million': 'O'},
  {'round': 'O'}]]